In [ ]:
!pip install transformers


In [ ]:
import torch

# Verificar si hay acceso a una GPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Crear un tensor en la CPU
tensor_cpu = torch.tensor([1, 2, 3])

# Enviar el tensor a la GPU
tensor_gpu = tensor_cpu.to(device)

# Realizar cálculos en la GPU
resultado_gpu = tensor_gpu * 2

# Obtener el resultado de nuevo en la CPU si es necesario
resultado_cpu = resultado_gpu.to("cpu")

# Imprimir el resultado
print("Resultado en la CPU:", resultado_cpu)


In [ ]:
import pandas as pd
import spacy
!python -m spacy download es_core_news_sm


In [ ]:
# Cargar los datos de entrenamiento
data = pd.read_csv('SMM4H_2024_Task3_Training_1800.csv', nrows=1800, usecols=[0, 1, 2, 3], engine='python')

# Cargar el modelo de spaCy para tokenizar oraciones
nlp = spacy.load("es_core_news_sm")

# Filtrar el DataFrame para quedarse solo con las filas donde 'label' es diferente de 0
data_filtered = data.loc[data['label'] != 0]

# Función para obtener los enunciados con las keywords
def extract_keyword_sentences(row):
    keywords = set(row['keyword'].split(', '))  # Convertir las keywords en un conjunto para eliminar duplicados
    text = row['text']
    doc = nlp(text)
    found_sentences = set()  # Conjunto para almacenar las oraciones encontradas
    for sentence in doc.sents:
        # Verificar si la oración contiene al menos una de las keywords
        if any(keyword.lower() in sentence.text.lower() for keyword in keywords):
            found_sentences.add(sentence.text.strip())  # Agregar la oración al conjunto de oraciones encontradas
    return '. '.join(found_sentences) if found_sentences else None  # Unir las oraciones encontradas en un solo texto

# Filtrar y actualizar el texto con los enunciados que contienen las keywords
data_filtered['text'] = data_filtered.apply(lambda row: extract_keyword_sentences(row), axis=1)
datatext = data_filtered['text'].astype(str)
print(len(datatext))
print(datatext[0:10])


669
1131    But at any rate, my advice to you is to do wha...
1132    I'm gonna do the Pokemon thing to get myself t...
1133    Taking a walk is a great way, you can start in...
1134    Your son doesn't necessarily need to go to the...
1135    I use to go running at this track in the field...
1136    no need to go gym that makes me nervous walkin...
1137    The winter means a lot of quite time, sitting ...
1138    I have been contemplating ever since the last ...
1139                          Just go on a walk to relax.
1140    I feel really awkward eating around people at ...
Name: text, dtype: object


<ipython-input-20-b5adf5eea333>:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_filtered['text'] = data_filtered.apply(lambda row: extract_keyword_sentences(row), axis=1)




---



## **MODEL: cardiffnlp/twitter-roberta-base-sentiment**

https://huggingface.co/cardiffnlp/twitter-roberta-base-sentiment

In [ ]:
from transformers import pipeline, AutoModelForSequenceClassification, AutoTokenizer

task='sentiment'
MODEL = f"cardiffnlp/twitter-roberta-base-{task}"
tokenizer = AutoTokenizer.from_pretrained(MODEL)
model = AutoModelForSequenceClassification.from_pretrained(MODEL)


In [ ]:
# Crear el pipeline utilizando el modelo y el tokenizer
sentiment_task = pipeline('sentiment-analysis', model=model, tokenizer=tokenizer)

# Convertir cada texto en el formato deseado y hacer las predicciones
formatted_texts = datatext.apply(lambda text: f"('{text}')")
print(len(formatted_texts))


669


In [ ]:
# Convertir cada texto en el formato deseado y hacer las predicciones
predictions_5 = []
x = 0
for i, text in enumerate(formatted_texts):
    try:
        # Tokenizar el texto para obtener los token_type_ids
        inputs = tokenizer(text, return_tensors="pt", padding=True, truncation=True, max_length=512)
        # Extraer la cadena de texto del tensor de entrada
        text_input = tokenizer.decode(inputs["input_ids"][0], skip_special_tokens=True)
        # Hacer la predicción con la cadena de texto
        output_5 = sentiment_task(text_input)
        # Obtener la etiqueta de la predicción
        label = output_5[0]['label']
        # Convertir la etiqueta a su valor numérico
        if label == 'LABEL_2':
            predictions_5.append(1)
        elif label == 'LABEL_1':
            predictions_5.append(2)
        else:
            predictions_5.append(3)
    except Exception as e:
        print(f"Error al procesar texto {text}: {e}")
        # Asignar un valor de 3 a la predicción en caso de error
        predictions_5.append(3)
    x += 1
    print(f"Texto {x}: {output_5}")

print(predictions_5)


Texto 1: [{'label': 'LABEL_2', 'score': 0.6811107397079468}]
Texto 2: [{'label': 'LABEL_2', 'score': 0.9822790622711182}]
Texto 3: [{'label': 'LABEL_0', 'score': 0.4537239968776703}]
Texto 4: [{'label': 'LABEL_1', 'score': 0.4409448802471161}]
Texto 5: [{'label': 'LABEL_0', 'score': 0.6242709159851074}]
Texto 6: [{'label': 'LABEL_1', 'score': 0.5730252861976624}]
Texto 7: [{'label': 'LABEL_1', 'score': 0.38199880719184875}]
Texto 8: [{'label': 'LABEL_0', 'score': 0.8162810206413269}]
Texto 9: [{'label': 'LABEL_0', 'score': 0.46096372604370117}]
Texto 10: [{'label': 'LABEL_0', 'score': 0.7482565641403198}]
Texto 11: [{'label': 'LABEL_2', 'score': 0.9899139404296875}]
Texto 12: [{'label': 'LABEL_2', 'score': 0.9811497926712036}]
Texto 13: [{'label': 'LABEL_2', 'score': 0.7175377607345581}]
Texto 14: [{'label': 'LABEL_1', 'score': 0.4933836758136749}]
Texto 15: [{'label': 'LABEL_2', 'score': 0.5123746395111084}]
Texto 16: [{'label': 'LABEL_1', 'score': 0.3985620439052582}]
Texto 17: [{'la

In [ ]:
from sklearn.metrics import classification_report

# Obtener las etiquetas originales del conjunto de datos
y_true = data['label'].iloc[-669:]

# Calcular el conteo de cada clase
class_counts = {label: predictions_5.count(label) for label in set(predictions_5)}

# Mostrar el conteo de cada clase
print("Conteo de cada clase:")
for label, count in class_counts.items():
    print(f"Clase {label}: {count} instancias")

# Crear el classification_report con las etiquetas originales del DataFrame 'data'
print("\nClassification Report:")
print(classification_report(y_true, predictions_5))


Conteo de cada clase:
Clase 1: 158 instancias
Clase 2: 144 instancias
Clase 3: 367 instancias

Classification Report:
              precision    recall  f1-score   support

           1       0.51      0.50      0.50       160
           2       0.59      0.22      0.32       395
           3       0.28      0.89      0.42       114

    accuracy                           0.40       669
   macro avg       0.46      0.53      0.41       669
weighted avg       0.52      0.40      0.38       669




---



## **MODEL: pysentimiento/robertuito-sentiment-analysis**

https://huggingface.co/pysentimiento/robertuito-sentiment-analysis

In [ ]:
from transformers import pipeline

# Crear el pipeline para el análisis de sentimientos
sentiment_task = pipeline('sentiment-analysis', model='pysentimiento/robertuito-sentiment-analysis', tokenizer='pysentimiento/robertuito-sentiment-analysis', truncation=True, max_length=512)

# Convertir cada texto en el formato deseado y hacer las predicciones
formatted_texts = datatext.apply(lambda text: f"('{text}')")
print(len(formatted_texts))


669


In [ ]:
# Convertir cada texto en el formato deseado y hacer las predicciones
predictions_5 = []
x = 0
for i, text in enumerate(formatted_texts):
    try:
        # Hacer la predicción con el texto
        output_5 = sentiment_task(text)
        # Obtener la etiqueta de la predicción
        label = output_5[0]['label']
        # Convertir la etiqueta a su valor numérico
        if label == 'POS':
            predictions_5.append(1)
        elif label == 'NEU':
            predictions_5.append(2)
        else:
            predictions_5.append(3)
        print(f"Texto {x}: {output_5}")
    except Exception as e:
        print(f"Error al procesar texto {text}: {e}")
        # Asignar un valor de 3 a la predicción en caso de error
        predictions_5.append(3)
    x += 1

print(predictions_5)

Texto 0: [{'label': 'NEU', 'score': 0.5072993040084839}]
Texto 1: [{'label': 'NEU', 'score': 0.6313481330871582}]
Texto 2: [{'label': 'POS', 'score': 0.8258228898048401}]
Texto 3: [{'label': 'NEU', 'score': 0.49526265263557434}]
Error al procesar texto ('I use to go running at this track in the field of a school early in the morning like 6:30.. Running in the daytime is just the worst so many people.. I even walk way faster then normal.. and this is weird but its almost hard for me to slow down, I feel like I'm galloping like a horse or something.. Also to make it worse I run really fast which I feel draws a lot of attention to me.. But now that school has started, not only can't I run there but theres like the fear of morning hustle  and bustle of parents dropping off kids no matter where I run.'): index out of range in self
Texto 5: [{'label': 'NEU', 'score': 0.5681695938110352}]
Texto 6: [{'label': 'NEU', 'score': 0.5374426245689392}]
Error al procesar texto ('I have been contemplat

In [ ]:
from sklearn.metrics import classification_report

# Obtener las etiquetas originales del conjunto de datos
y_true = data['label'].iloc[-669:]
print(y_true)

# Calcular el conteo de cada clase
class_counts = {label: predictions_5.count(label) for label in set(predictions_5)}

# Mostrar el conteo de cada clase
print("Conteo de cada clase:")
for label, count in class_counts.items():
    print(f"Clase {label}: {count} instancias")

# Crear el classification_report con las etiquetas originales del DataFrame 'data'
print("\nClassification Report:")
print(classification_report(y_true, predictions_5))

1131    1
1132    1
1133    1
1134    1
1135    1
       ..
1795    3
1796    3
1797    3
1798    3
1799    3
Name: label, Length: 669, dtype: int64
Conteo de cada clase:
Clase 1: 87 instancias
Clase 2: 304 instancias
Clase 3: 278 instancias

Classification Report:
              precision    recall  f1-score   support

           1       0.47      0.26      0.33       160
           2       0.60      0.46      0.52       395
           3       0.30      0.74      0.43       114

    accuracy                           0.46       669
   macro avg       0.46      0.48      0.43       669
weighted avg       0.52      0.46      0.46       669






---



## **MODEL: mrm8488/distilroberta-finetuned-financial-news-sentiment-analysis**


https://huggingface.co/mrm8488/distilroberta-finetuned-financial-news-sentiment-analysis?text=Something+that+work+for+me+is+to+expose+myself+to+my+anxiety.+Even+though+I%27m+starting+to+become+anxious%2C+I+keep+doing+it.+The+trick+is+to+not+go+over+a+threshold.+If+I%27m+too+anxious%2C+it+affect+how+I+act+and+then+everything+goes+bad+%28not+that+bad+but+my+anxiety+amplify+everything+that+goes+bad+and+then+I+keep+thinking+about+it+and+it+just+worse%29.+Taking+a+walk+is+a+great+way%2C+you+can+start+in+a+residential+place+with+less+people+and+then+move+on+to+place+with+more+and+more.+Going+to+movie+theaters%2C+fast-food%2C+restaurants%2C+conventions%2C+also+helped+me.+All+situations+where+I+had+just+a+little+anxiety+but+that+let+me+see+how+people+acts+and+let+me+understands+them+a+little+better.+Another+thing+that+you+need+to+do%2C+is+stopping+to+be+negative+about+yourself.+You+don%27t+seems+to+be+that+negative%2C+though+you+do+say+%22anyone+who+bothers+to+read+this%22+which+seems+like+it%27s+negative+reading+your+thread+%28it+doesn%27t+bother+us%2C+we+are+there+to+read+theses+threads%2C+like+any+other+one%29%2C+but+it%27s+important+that+you+are+aware+that+you+should+avoid+negative+though.+Often+we+justify+being+negative+by+saying+that+this+way+you+are+aware+of+your+flaw%2C+which+is+better+than+not.+Sadly+often+we+are+wrong%2C+theses+negatives+though+aren%27t+that+important+and+they+are+amazing+to+lose+any+motivation+to+improve.+Instead%2C+you+need+to+find+the+positive+in+the+situation+and+if+really+there%27s+something+negative+important%2C+don%27t+see+it+as+negative+but+as+something+to+improve.++at+this+point+i+have+no+assurances+and+putting+my+faith+in+a+psychologist+only+to+go+thru+months+and+years+of+no+improvement+would+probably+destroy+me.+It+doesn%27t+destroy+you+right+now%3F+Really+how+trying+anything+is+worst+than+not+trying+anything+and+staying+the+mess+you+are+right+now%3F+Believe+me%2C+you+can%27t+be+worst+than+you+are+right+now%2C+you+can+fail%2C+and+you+may+fail+often%2C+but+that+will+just+bring+you+back+to+where+you+are+right+now.+Anxiety+is+a+bitch+though+and+it+will+always+try+to+give+you+the+impression+you+don%27t+improve+and+are+just+worst+than+you+ever+was+by+simply+trying.+You+need+to+stay+positive.

In [ ]:
from transformers import pipeline

# Cargar el modelo de análisis de sentimientos
sentiment_task = pipeline("sentiment-analysis", model="mrm8488/distilroberta-finetuned-financial-news-sentiment-analysis", truncation=True, max_length=512)


In [ ]:
data_filtered_all_text = data.loc[data['label'] != 0]
data_all_text = data_filtered_all_text['text']

# Convertir cada texto en el formato deseado y hacer las predicciones
formatted_texts_all_text = data_all_text.apply(lambda text: f"('{text}')")
print(len(formatted_texts_all_text))
print(formatted_texts_all_text[0:1])

669
1131    (' Do you feel like the texts that you send ba...
Name: text, dtype: object


In [ ]:
# Convertir cada texto en el formato deseado y hacer las predicciones
predictions_50 = []
x = 0
for i, text in enumerate(formatted_texts_all_text):
    try:
        # Hacer la predicción con el texto
        output_50 = sentiment_task(text)
        # Obtener la etiqueta de la predicción
        label = output_50[0]['label']
        # Convertir la etiqueta a su valor numérico
        if label == 'positive':
            predictions_50.append(1)
        elif label == 'neutral':
            predictions_50.append(2)
        else:
            predictions_50.append(3)
        print(f"Texto {x}: {output_50}")
    except Exception as e:
        print(f"Error al procesar texto {text}: {e}")
        # Asignar un valor de 3 a la predicción en caso de error
        predictions_50.append(3)
    x += 1

print(predictions_50)


Texto 0: [{'label': 'neutral', 'score': 0.9988760352134705}]
Texto 1: [{'label': 'neutral', 'score': 0.9665501713752747}]
Texto 2: [{'label': 'neutral', 'score': 0.9983677268028259}]
Texto 3: [{'label': 'positive', 'score': 0.8254127502441406}]
Texto 4: [{'label': 'positive', 'score': 0.9803810715675354}]
Texto 5: [{'label': 'neutral', 'score': 0.9973171353340149}]
Texto 6: [{'label': 'neutral', 'score': 0.9992883801460266}]
Texto 7: [{'label': 'neutral', 'score': 0.8780035376548767}]
Texto 8: [{'label': 'neutral', 'score': 0.9687804579734802}]
Texto 9: [{'label': 'neutral', 'score': 0.9528299570083618}]
Texto 10: [{'label': 'positive', 'score': 0.9984115362167358}]
Texto 11: [{'label': 'positive', 'score': 0.9888294339179993}]
Texto 12: [{'label': 'positive', 'score': 0.9667387008666992}]
Texto 13: [{'label': 'neutral', 'score': 0.9994632601737976}]
Texto 14: [{'label': 'neutral', 'score': 0.9970719814300537}]
Texto 15: [{'label': 'negative', 'score': 0.460610032081604}]
Texto 16: [{'

In [ ]:
from sklearn.metrics import classification_report

# Obtener las etiquetas originales del conjunto de datos
y_true = data['label'].iloc[-669:]
print(y_true)

# Calcular el conteo de cada clase
class_counts = {label: predictions_50.count(label) for label in set(predictions_50)}

# Mostrar el conteo de cada clase
print("Conteo de cada clase:")
for label, count in class_counts.items():
    print(f"Clase {label}: {count} instancias")

# Crear el classification_report con las etiquetas originales del DataFrame 'data'
print("\nClassification Report:")
print(classification_report(y_true, predictions_50))

1131    1
1132    1
1133    1
1134    1
1135    1
       ..
1795    3
1796    3
1797    3
1798    3
1799    3
Name: label, Length: 669, dtype: int64
Conteo de cada clase:
Clase 1: 64 instancias
Clase 2: 507 instancias
Clase 3: 98 instancias

Classification Report:
              precision    recall  f1-score   support

           1       0.58      0.23      0.33       160
           2       0.63      0.81      0.71       395
           3       0.39      0.33      0.36       114

    accuracy                           0.59       669
   macro avg       0.53      0.46      0.47       669
weighted avg       0.58      0.59      0.56       669





---



## **MODEL: finiteautomata/beto-sentiment-analysis**

https://huggingface.co/finiteautomata/beto-sentiment-analysis?text=Do+you+feel+like+the+texts+that+you+send+back+and+forth+are+pretty+flirtatious%3F+I+think+that+would+be+a+good+way+to+gauge+whether+or+not+he+has+similar+feelings+towards+you.+In+terms+of+talking+to+him+in+person%2C+you+don%27t+necessarily+have+to+give+off+the+perfect+impression+every+time+you+are+with+him.+If+he+is+a+genuine+guy%2C+he+won%27t+judge+you+for+appearing+a+little+shy+or+vulnerable.+But+at+any+rate%2C+my+advice+to+you+is+to+do+whatever+it+is+that+calms+you+down+%28listen+to+music%2C+take+a+short+walk%2C+etc.%29+and+just+go+for+it.+Whether+he+responds+positively+to+you+or+not%2C+you+will+never+know+unless+you+try.+Also%2C+don%27t+try+and+force+yourself+to+act+any+differently+and+be+true+to+yourself.+This+is+all+pretty+cliche%2C+but+it+really+is+the+truth+haha%2C+and+the+older+I+get%2C+the+more+I+realize+it.+But+yeah%2C+just+be+yourself+and+give+it+a+shot%2C+you+won%27t+get+a+second+chance+if+you+don%27t+do+it+now+so+go+for+it%21+I+hope+I+was+able+to+help+a+little+and+good+luck%21%21

In [ ]:
from transformers import pipeline, AutoTokenizer

# Definir el modelo y el tokenizador
model_name = "finiteautomata/beto-sentiment-analysis"
tokenizer = AutoTokenizer.from_pretrained(model_name)
sentiment_task = pipeline("sentiment-analysis", model=model_name, tokenizer=tokenizer, truncation=True, max_length=512)


In [ ]:
# Convertir cada texto en el formato deseado y hacer las predicciones
predictions_51 = []
x = 0
for i, text in enumerate(formatted_texts_all_text):
    try:
        # Hacer la predicción con el texto
        output_51 = sentiment_task(text)
        # Obtener la etiqueta de la predicción
        label = output_51[0]['label']
        # Convertir la etiqueta a su valor numérico
        if label == 'POS':
            predictions_51.append(1)
        elif label == 'NEU':
            predictions_51.append(2)
        else:
            predictions_51.append(3)
        print(f"Texto {x}: {output_51}")
    except Exception as e:
        print(f"Error al procesar texto {text}: {e}")
        # Asignar un valor de 3 a la predicción en caso de error
        predictions_51.append(3)
    x += 1

print(predictions_51)


Texto 0: [{'label': 'NEU', 'score': 0.5491818785667419}]
Texto 1: [{'label': 'POS', 'score': 0.9953455328941345}]
Texto 2: [{'label': 'NEG', 'score': 0.9765090346336365}]
Texto 3: [{'label': 'NEU', 'score': 0.6010398268699646}]
Texto 4: [{'label': 'NEG', 'score': 0.7144289612770081}]
Texto 5: [{'label': 'NEG', 'score': 0.9975156784057617}]
Texto 6: [{'label': 'NEU', 'score': 0.8381280303001404}]
Texto 7: [{'label': 'NEG', 'score': 0.5983966588973999}]
Texto 8: [{'label': 'NEU', 'score': 0.7156984806060791}]
Texto 9: [{'label': 'NEU', 'score': 0.9620773196220398}]
Texto 10: [{'label': 'POS', 'score': 0.9960836172103882}]
Texto 11: [{'label': 'POS', 'score': 0.9721501469612122}]
Texto 12: [{'label': 'NEG', 'score': 0.6158750653266907}]
Texto 13: [{'label': 'NEU', 'score': 0.4533942639827728}]
Texto 14: [{'label': 'NEU', 'score': 0.9932553768157959}]
Texto 15: [{'label': 'POS', 'score': 0.4851565361022949}]
Texto 16: [{'label': 'NEU', 'score': 0.9331086874008179}]
Texto 17: [{'label': 'NE

In [ ]:
from sklearn.metrics import classification_report

# Obtener las etiquetas originales del conjunto de datos
y_true = data['label'].iloc[-669:]
print(y_true)

# Calcular el conteo de cada clase
class_counts = {label: predictions_51.count(label) for label in set(predictions_51)}

# Mostrar el conteo de cada clase
print("Conteo de cada clase:")
for label, count in class_counts.items():
    print(f"Clase {label}: {count} instancias")

# Crear el classification_report con las etiquetas originales del DataFrame 'data'
print("\nClassification Report:")
print(classification_report(y_true, predictions_51))

1131    1
1132    1
1133    1
1134    1
1135    1
       ..
1795    3
1796    3
1797    3
1798    3
1799    3
Name: label, Length: 669, dtype: int64
Conteo de cada clase:
Clase 1: 51 instancias
Clase 2: 406 instancias
Clase 3: 212 instancias

Classification Report:
              precision    recall  f1-score   support

           1       0.45      0.14      0.22       160
           2       0.61      0.63      0.62       395
           3       0.27      0.51      0.36       114

    accuracy                           0.49       669
   macro avg       0.45      0.43      0.40       669
weighted avg       0.52      0.49      0.48       669





---



## **MODEL: ahmedrachid/FinancialBERT-Sentiment-Analysis**

https://huggingface.co/ahmedrachid/FinancialBERT-Sentiment-Analysis?text=Something+that+work+for+me+is+to+expose+myself+to+my+anxiety.+Even+though+I%27m+starting+to+become+anxious%2C+I+keep+doing+it.+The+trick+is+to+not+go+over+a+threshold.+If+I%27m+too+anxious%2C+it+affect+how+I+act+and+then+everything+goes+bad+%28not+that+bad+but+my+anxiety+amplify+everything+that+goes+bad+and+then+I+keep+thinking+about+it+and+it+just+worse%29.+Taking+a+walk+is+a+great+way%2C+you+can+start+in+a+residential+place+with+less+people+and+then+move+on+to+place+with+more+and+more.+Going+to+movie+theaters%2C+fast-food%2C+restaurants%2C+conventions%2C+also+helped+me.+All+situations+where+I+had+just+a+little+anxiety+but+that+let+me+see+how+people+acts+and+let+me+understands+them+a+little+better.+Another+thing+that+you+need+to+do%2C+is+stopping+to+be+negative+about+yourself.+You+don%27t+seems+to+be+that+negative%2C+though+you+do+say+%22anyone+who+bothers+to+read+this%22+which+seems+like+it%27s+negative+reading+your+thread+%28it+doesn%27t+bother+us%2C+we+are+there+to+read+theses+threads%2C+like+any+other+one%29%2C+but+it%27s+important+that+you+are+aware+that+you+should+avoid+negative+though.+Often+we+justify+being+negative+by+saying+that+this+way+you+are+aware+of+your+flaw%2C+which+is+better+than+not.+Sadly+often+we+are+wrong%2C+theses+negatives+though+aren%27t+that+important+and+they+are+amazing+to+lose+any+motivation+to+improve.+Instead%2C+you+need+to+find+the+positive+in+the+situation+and+if+really+there%27s+something+negative+important%2C+don%27t+see+it+as+negative+but+as+something+to+improve.++at+this+point+i+have+no+assurances+and+putting+my+faith+in+a+psychologist+only+to+go+thru+months+and+years+of+no+improvement+would+probably+destroy+me.+It+doesn%27t+destroy+you+right+now%3F+Really+how+trying+anything+is+worst+than+not+trying+anything+and+staying+the+mess+you+are+right+now%3F+Believe+me%2C+you+can%27t+be+worst+than+you+are+right+now%2C+you+can+fail%2C+and+you+may+fail+often%2C+but+that+will+just+bring+you+back+to+where+you+are+right+now.+Anxiety+is+a+bitch+though+and+it+will+always+try+to+give+you+the+impression+you+don%27t+improve+and+are+just+worst+than+you+ever+was+by+simply+trying.+You+need+to+stay+positive.

In [ ]:
from transformers import pipeline, AutoTokenizer

# Definir el modelo y el tokenizador
model_name = "ahmedrachid/FinancialBERT-Sentiment-Analysis"
tokenizer = AutoTokenizer.from_pretrained(model_name)
sentiment_task = pipeline("sentiment-analysis", model=model_name, tokenizer=tokenizer, truncation=True, max_length=512)


tokenizer_config.json:   0%|          | 0.00/369 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/226k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/464k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/789 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/439M [00:00<?, ?B/s]

In [ ]:
# Convertir cada texto en el formato deseado y hacer las predicciones
predictions_52 = []
x = 0
for i, text in enumerate(formatted_texts_all_text):
    try:
        # Hacer la predicción con el texto
        output_52 = sentiment_task(text)
        # Obtener la etiqueta de la predicción
        label = output_52[0]['label']
        # Convertir la etiqueta a su valor numérico
        if label == 'positive':
            predictions_52.append(1)
        elif label == 'neutral':
            predictions_52.append(2)
        else:
            predictions_52.append(3)
        print(f"Texto {x}: {output_52}")
    except Exception as e:
        print(f"Error al procesar texto {text}: {e}")
        # Asignar un valor de 3 a la predicción en caso de error
        predictions_52.append(3)
    x += 1

print(predictions_52)


Texto 0: [{'label': 'neutral', 'score': 0.9965232610702515}]
Texto 1: [{'label': 'neutral', 'score': 0.6423987746238708}]
Texto 2: [{'label': 'neutral', 'score': 0.9860018491744995}]
Texto 3: [{'label': 'neutral', 'score': 0.9985532164573669}]
Texto 4: [{'label': 'neutral', 'score': 0.9784753322601318}]
Texto 5: [{'label': 'neutral', 'score': 0.9992094039916992}]
Texto 6: [{'label': 'neutral', 'score': 0.9942639470100403}]
Texto 7: [{'label': 'neutral', 'score': 0.9932547211647034}]
Texto 8: [{'label': 'neutral', 'score': 0.9956998825073242}]
Texto 9: [{'label': 'neutral', 'score': 0.9986376166343689}]
Texto 10: [{'label': 'positive', 'score': 0.927489161491394}]
Texto 11: [{'label': 'neutral', 'score': 0.998173713684082}]
Texto 12: [{'label': 'positive', 'score': 0.7923166155815125}]
Texto 13: [{'label': 'neutral', 'score': 0.9902021288871765}]
Texto 14: [{'label': 'neutral', 'score': 0.9959206581115723}]
Texto 15: [{'label': 'neutral', 'score': 0.981025755405426}]
Texto 16: [{'label'

In [ ]:
from sklearn.metrics import classification_report

# Obtener las etiquetas originales del conjunto de datos
y_true = data['label'].iloc[-669:]
print(y_true)

# Calcular el conteo de cada clase
class_counts = {label: predictions_52.count(label) for label in set(predictions_52)}

# Mostrar el conteo de cada clase
print("Conteo de cada clase:")
for label, count in class_counts.items():
    print(f"Clase {label}: {count} instancias")

# Crear el classification_report con las etiquetas originales del DataFrame 'data'
print("\nClassification Report:")
print(classification_report(y_true, predictions_52))

1131    1
1132    1
1133    1
1134    1
1135    1
       ..
1795    3
1796    3
1797    3
1798    3
1799    3
Name: label, Length: 669, dtype: int64
Conteo de cada clase:
Clase 1: 11 instancias
Clase 2: 648 instancias
Clase 3: 10 instancias

Classification Report:
              precision    recall  f1-score   support

           1       0.64      0.04      0.08       160
           2       0.59      0.97      0.74       395
           3       0.30      0.03      0.05       114

    accuracy                           0.59       669
   macro avg       0.51      0.35      0.29       669
weighted avg       0.55      0.59      0.46       669





---



## **MODEL: finiteautomata/bertweet-base-sentiment-analysis**

https://huggingface.co/finiteautomata/bertweet-base-sentiment-analysis


In [ ]:
from transformers import pipeline, AutoTokenizer

# Definir el modelo y el tokenizador
model_name = "finiteautomata/bertweet-base-sentiment-analysis"
tokenizer = AutoTokenizer.from_pretrained(model_name)
sentiment_task = pipeline("sentiment-analysis", model=model_name, tokenizer=tokenizer)


tokenizer_config.json:   0%|          | 0.00/338 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/843k [00:00<?, ?B/s]

bpe.codes:   0%|          | 0.00/1.08M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/22.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/167 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/949 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/540M [00:00<?, ?B/s]

In [ ]:
# Convertir cada texto en el formato deseado y hacer las predicciones
predictions_53 = []
x = 0
for i, text in enumerate(formatted_texts):
    try:
        # Hacer la predicción con el texto
        output_53 = sentiment_task(text)
        # Obtener la etiqueta de la predicción
        label = output_53[0]['label']
        # Convertir la etiqueta a su valor numérico
        if label == 'POS':
            predictions_53.append(1)
        elif label == 'NEU':
            predictions_53.append(2)
        else:
            predictions_53.append(3)
        print(f"Texto {x}: {output_53}")
    except Exception as e:
        print(f"Error al procesar texto {text}: {e}")
        # Asignar un valor de 3 a la predicción en caso de error
        predictions_53.append(3)
    x += 1

print(predictions_53)


Texto 0: [{'label': 'NEU', 'score': 0.586448073387146}]
Texto 1: [{'label': 'NEU', 'score': 0.5483405590057373}]
Texto 2: [{'label': 'POS', 'score': 0.983121395111084}]
Texto 3: [{'label': 'NEU', 'score': 0.7703757286071777}]
Error al procesar texto ('I use to go running at this track in the field of a school early in the morning like 6:30.. Running in the daytime is just the worst so many people.. I even walk way faster then normal.. and this is weird but its almost hard for me to slow down, I feel like I'm galloping like a horse or something.. Also to make it worse I run really fast which I feel draws a lot of attention to me.. But now that school has started, not only can't I run there but theres like the fear of morning hustle  and bustle of parents dropping off kids no matter where I run.'): index out of range in self
Texto 5: [{'label': 'NEU', 'score': 0.862480878829956}]
Texto 6: [{'label': 'POS', 'score': 0.8567403554916382}]
Texto 7: [{'label': 'NEG', 'score': 0.87454450130462

In [ ]:
from sklearn.metrics import classification_report

# Obtener las etiquetas originales del conjunto de datos
y_true = data['label'].iloc[-669:]
print(y_true)

# Calcular el conteo de cada clase
class_counts = {label: predictions_53.count(label) for label in set(predictions_53)}

# Mostrar el conteo de cada clase
print("Conteo de cada clase:")
for label, count in class_counts.items():
    print(f"Clase {label}: {count} instancias")

# Crear el classification_report con las etiquetas originales del DataFrame 'data'
print("\nClassification Report:")
print(classification_report(y_true, predictions_53))

1131    1
1132    1
1133    1
1134    1
1135    1
       ..
1795    3
1796    3
1797    3
1798    3
1799    3
Name: label, Length: 669, dtype: int64
Conteo de cada clase:
Clase 1: 129 instancias
Clase 2: 282 instancias
Clase 3: 258 instancias

Classification Report:
              precision    recall  f1-score   support

           1       0.46      0.37      0.41       160
           2       0.62      0.44      0.51       395
           3       0.31      0.71      0.44       114

    accuracy                           0.47       669
   macro avg       0.46      0.51      0.45       669
weighted avg       0.53      0.47      0.48       669



---



## **MODEL: mr8488/distilroberta-finetuned-financial-news-sentiment-analysis-v2**

https://huggingface.co/mr8488/distilroberta-finetuned-financial-news-sentiment-analysis-v2

In [ ]:
from transformers import pipeline

# Cargar el modelo de análisis de sentimientos
sentiment_task = pipeline("sentiment-analysis", model="mr8488/distilroberta-finetuned-financial-news-sentiment-analysis-v2", truncation=True, max_length=512)


config.json:   0%|          | 0.00/980 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/328M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.20k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/999k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/964 [00:00<?, ?B/s]

In [ ]:
# Convertir cada texto en el formato deseado y hacer las predicciones
predictions_54 = []
x = 0
for i, text in enumerate(formatted_texts_all_text):
    try:
        # Hacer la predicción con el texto
        output_54 = sentiment_task(text)
        # Obtener la etiqueta de la predicción
        label = output_54[0]['label']
        # Convertir la etiqueta a su valor numérico
        if label == 'positive':
            predictions_54.append(1)
        elif label == 'neutral':
            predictions_54.append(2)
        else:
            predictions_54.append(3)
        print(f"Texto {x}: {output_54}")
    except Exception as e:
        print(f"Error al procesar texto {text}: {e}")
        # Asignar un valor de 3 a la predicción en caso de error
        predictions_54.append(3)
    x += 1

print(predictions_54)


Texto 0: [{'label': 'neutral', 'score': 0.9137879610061646}]
Texto 1: [{'label': 'neutral', 'score': 0.8186712265014648}]
Texto 2: [{'label': 'neutral', 'score': 0.9147228598594666}]
Texto 3: [{'label': 'neutral', 'score': 0.8623017072677612}]
Texto 4: [{'label': 'neutral', 'score': 0.49183574318885803}]
Texto 5: [{'label': 'neutral', 'score': 0.9181729555130005}]
Texto 6: [{'label': 'neutral', 'score': 0.902207612991333}]
Texto 7: [{'label': 'neutral', 'score': 0.5468345284461975}]
Texto 8: [{'label': 'neutral', 'score': 0.7737430334091187}]
Texto 9: [{'label': 'neutral', 'score': 0.5763322710990906}]
Texto 10: [{'label': 'positive', 'score': 0.5851956009864807}]
Texto 11: [{'label': 'neutral', 'score': 0.6348981261253357}]
Texto 12: [{'label': 'neutral', 'score': 0.7331238985061646}]
Texto 13: [{'label': 'neutral', 'score': 0.877716600894928}]
Texto 14: [{'label': 'neutral', 'score': 0.9036850929260254}]
Texto 15: [{'label': 'neutral', 'score': 0.7600439786911011}]
Texto 16: [{'label

In [ ]:
from sklearn.metrics import classification_report

# Obtener las etiquetas originales del conjunto de datos
y_true = data['label'].iloc[-669:]
print(y_true)

# Calcular el conteo de cada clase
class_counts = {label: predictions_54.count(label) for label in set(predictions_54)}

# Mostrar el conteo de cada clase
print("Conteo de cada clase:")
for label, count in class_counts.items():
    print(f"Clase {label}: {count} instancias")

# Crear el classification_report con las etiquetas originales del DataFrame 'data'
print("\nClassification Report:")
print(classification_report(y_true, predictions_54))

1131    1
1132    1
1133    1
1134    1
1135    1
       ..
1795    3
1796    3
1797    3
1798    3
1799    3
Name: label, Length: 669, dtype: int64
Conteo de cada clase:
Clase 1: 7 instancias
Clase 2: 561 instancias
Clase 3: 101 instancias

Classification Report:
              precision    recall  f1-score   support

           1       0.71      0.03      0.06       160
           2       0.60      0.86      0.71       395
           3       0.39      0.34      0.36       114

    accuracy                           0.57       669
   macro avg       0.57      0.41      0.38       669
weighted avg       0.59      0.57      0.49       669





---



## **MODEL: ayameRushia/bert-base-indonesian-1.5G-sentiment-analysis-smsa**

https://huggingface.co/ayameRushia/bert-base-indonesian-1.5G-sentiment-analysis-smsa?text=Something+that+work+for+me+is+to+expose+myself+to+my+anxiety.+Even+though+I%27m+starting+to+become+anxious%2C+I+keep+doing+it.+The+trick+is+to+not+go+over+a+threshold.+If+I%27m+too+anxious%2C+it+affect+how+I+act+and+then+everything+goes+bad+%28not+that+bad+but+my+anxiety+amplify+everything+that+goes+bad+and+then+I+keep+thinking+about+it+and+it+just+worse%29.+Taking+a+walk+is+a+great+way%2C+you+can+start+in+a+residential+place+with+less+people+and+then+move+on+to+place+with+more+and+more.+Going+to+movie+theaters%2C+fast-food%2C+restaurants%2C+conventions%2C+also+helped+me.+All+situations+where+I+had+just+a+little+anxiety+but+that+let+me+see+how+people+acts+and+let+me+understands+them+a+little+better.+Another+thing+that+you+need+to+do%2C+is+stopping+to+be+negative+about+yourself.+You+don%27t+seems+to+be+that+negative%2C+though+you+do+say+%22anyone+who+bothers+to+read+this%22+which+seems+like+it%27s+negative+reading+your+thread+%28it+doesn%27t+bother+us%2C+we+are+there+to+read+theses+threads%2C+like+any+other+one%29%2C+but+it%27s+important+that+you+are+aware+that+you+should+avoid+negative+though.+Often+we+justify+being+negative+by+saying+that+this+way+you+are+aware+of+your+flaw%2C+which+is+better+than+not.+Sadly+often+we+are+wrong%2C+theses+negatives+though+aren%27t+that+important+and+they+are+amazing+to+lose+any+motivation+to+improve.+Instead%2C+you+need+to+find+the+positive+in+the+situation+and+if+really+there%27s+something+negative+important%2C+don%27t+see+it+as+negative

In [ ]:
from transformers import pipeline, AutoTokenizer

# Definir el modelo y el tokenizador
model_name = "ayameRushia/bert-base-indonesian-1.5G-sentiment-analysis-smsa"
tokenizer = AutoTokenizer.from_pretrained(model_name)
sentiment_task = pipeline("sentiment-analysis", model=model_name, tokenizer=tokenizer, truncation=True, max_length=512)

tokenizer_config.json:   0%|          | 0.00/387 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/230k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/476k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/899 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/443M [00:00<?, ?B/s]

In [ ]:
# Convertir cada texto en el formato deseado y hacer las predicciones
predictions_55 = []
x = 0
for i, text in enumerate(formatted_texts_all_text):
    try:
        # Hacer la predicción con el texto
        output_55 = sentiment_task(text)
        # Obtener la etiqueta de la predicción
        label = output_55[0]['label']
        # Convertir la etiqueta a su valor numérico
        if label == 'Positive':
            predictions_55.append(1)
        elif label == 'Neutral':
            predictions_55.append(2)
        else:
            predictions_55.append(3)
        print(f"Texto {x}: {output_55}")
    except Exception as e:
        print(f"Error al procesar texto {text}: {e}")
        # Asignar un valor de 3 a la predicción en caso de error
        predictions_55.append(3)
    x += 1

print(predictions_55)


Texto 0: [{'label': 'Negative', 'score': 0.6466971635818481}]
Texto 1: [{'label': 'Positive', 'score': 0.962917149066925}]
Texto 2: [{'label': 'Positive', 'score': 0.6553751826286316}]
Texto 3: [{'label': 'Neutral', 'score': 0.43980637192726135}]
Texto 4: [{'label': 'Negative', 'score': 0.5696832537651062}]
Texto 5: [{'label': 'Neutral', 'score': 0.9911164045333862}]
Texto 6: [{'label': 'Positive', 'score': 0.9105584621429443}]
Texto 7: [{'label': 'Negative', 'score': 0.7692186236381531}]
Texto 8: [{'label': 'Positive', 'score': 0.49067384004592896}]
Texto 9: [{'label': 'Neutral', 'score': 0.9867048859596252}]
Texto 10: [{'label': 'Positive', 'score': 0.9935163259506226}]
Texto 11: [{'label': 'Positive', 'score': 0.9883646965026855}]
Texto 12: [{'label': 'Positive', 'score': 0.7838127017021179}]
Texto 13: [{'label': 'Positive', 'score': 0.7752561569213867}]
Texto 14: [{'label': 'Positive', 'score': 0.753965437412262}]
Texto 15: [{'label': 'Negative', 'score': 0.8339203000068665}]
Texto

In [ ]:
from sklearn.metrics import classification_report

# Obtener las etiquetas originales del conjunto de datos
y_true = data['label'].iloc[-669:]
print(y_true)

# Calcular el conteo de cada clase
class_counts = {label: predictions_55.count(label) for label in set(predictions_55)}

# Mostrar el conteo de cada clase
print("Conteo de cada clase:")
for label, count in class_counts.items():
    print(f"Clase {label}: {count} instancias")

# Crear el classification_report con las etiquetas originales del DataFrame 'data'
print("\nClassification Report:")
print(classification_report(y_true, predictions_55))

1131    1
1132    1
1133    1
1134    1
1135    1
       ..
1795    3
1796    3
1797    3
1798    3
1799    3
Name: label, Length: 669, dtype: int64
Conteo de cada clase:
Clase 1: 167 instancias
Clase 2: 174 instancias
Clase 3: 328 instancias

Classification Report:
              precision    recall  f1-score   support

           1       0.43      0.45      0.44       160
           2       0.68      0.30      0.41       395
           3       0.26      0.74      0.38       114

    accuracy                           0.41       669
   macro avg       0.46      0.50      0.41       669
weighted avg       0.55      0.41      0.41       669




---



## **MODEL: avichr/heBERT_sentiment_analysis**

https://huggingface.co/avichr/heBERT_sentiment_analysis?text=Something+that+work+for+me+is+to+expose+myself+to+my+anxiety.+Even+though+I%27m+starting+to+become+anxious%2C+I+keep+doing+it.+The+trick+is+to+not+go+over+a+threshold.+If+I%27m+too+anxious%2C+it+affect+how+I+act+and+then+everything+goes+bad+%28not+that+bad+but+my+anxiety+amplify+everything+that+goes+bad+and+then+I+keep+thinking+about+it+and+it+just+worse%29.+Taking+a+walk+is+a+great+way%2C+you+can+start+in+a+residential+place+with+less+people+and+then+move+on+to+place+with+more+and+more.+Going+to+movie+theaters%2C+fast-food%2C+restaurants%2C+conventions%2C+also+helped+me.+All+situations+where+I+had+just+a+little+anxiety+but+that+let+me+see+how+people+acts+and+let+me+understands+them+a+little+better.+Another+thing+that+you+need+to+do%2C+is+stopping+to+be+negative+about+yourself.+You+don%27t+seems+to+be+that+negative%2C+though+you+do+say+%22anyone+who+bothers+to+read+this%22+which+seems+like+it%27s+negative+reading+your+thread+%28it+doesn%27t+bother+us%2C+we+are+there+to+read+theses+threads%2C+like+any+other+one%29%2C+but+it%27s+important+that+you+are+aware+that+you+should+avoid+negative+though.

In [ ]:
from transformers import pipeline, AutoTokenizer

# Definir el modelo y el tokenizador
model_name = "avichr/heBERT_sentiment_analysis"
tokenizer = AutoTokenizer.from_pretrained(model_name)
sentiment_task = pipeline("sentiment-analysis", model=model_name, tokenizer=tokenizer, truncation=True, max_length=512)

config.json:   0%|          | 0.00/677 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/299k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/438M [00:00<?, ?B/s]

In [ ]:
# Convertir cada texto en el formato deseado y hacer las predicciones
predictions_56 = []
x = 0
for i, text in enumerate(formatted_texts_all_text):
    try:
        # Hacer la predicción con el texto
        output_56 = sentiment_task(text)
        # Obtener la etiqueta de la predicción
        label = output_56[0]['label']
        # Convertir la etiqueta a su valor numérico
        if label == 'positive':
            predictions_56.append(1)
        elif label == 'neutral':
            predictions_56.append(2)
        else:
            predictions_56.append(3)
        print(f"Texto {x}: {output_56}")
    except Exception as e:
        print(f"Error al procesar texto {text}: {e}")
        # Asignar un valor de 3 a la predicción en caso de error
        predictions_56.append(3)
    x += 1

print(predictions_56)


Texto 0: [{'label': 'negative', 'score': 0.9929004907608032}]
Texto 1: [{'label': 'negative', 'score': 0.9362804889678955}]
Texto 2: [{'label': 'negative', 'score': 0.9955892562866211}]
Texto 3: [{'label': 'negative', 'score': 0.9962521195411682}]
Texto 4: [{'label': 'negative', 'score': 0.9895303249359131}]
Texto 5: [{'label': 'negative', 'score': 0.9997710585594177}]
Texto 6: [{'label': 'negative', 'score': 0.9962102174758911}]
Texto 7: [{'label': 'negative', 'score': 0.9940575361251831}]
Texto 8: [{'label': 'negative', 'score': 0.9781515598297119}]
Texto 9: [{'label': 'negative', 'score': 0.9995878338813782}]
Texto 10: [{'label': 'negative', 'score': 0.9987572431564331}]
Texto 11: [{'label': 'negative', 'score': 0.9994625449180603}]
Texto 12: [{'label': 'negative', 'score': 0.9969499707221985}]
Texto 13: [{'label': 'negative', 'score': 0.9555631279945374}]
Texto 14: [{'label': 'negative', 'score': 0.9997867941856384}]
Texto 15: [{'label': 'negative', 'score': 0.9575438499450684}]
Te

In [ ]:
from sklearn.metrics import classification_report

# Obtener las etiquetas originales del conjunto de datos
y_true = data['label'].iloc[-669:]
print(y_true)

# Calcular el conteo de cada clase
class_counts = {label: predictions_56.count(label) for label in set(predictions_56)}

# Mostrar el conteo de cada clase
print("Conteo de cada clase:")
for label, count in class_counts.items():
    print(f"Clase {label}: {count} instancias")

# Crear el classification_report con las etiquetas originales del DataFrame 'data'
print("\nClassification Report:")
print(classification_report(y_true, predictions_56))

1131    1
1132    1
1133    1
1134    1
1135    1
       ..
1795    3
1796    3
1797    3
1798    3
1799    3
Name: label, Length: 669, dtype: int64
Conteo de cada clase:
Clase 1: 1 instancias
Clase 2: 9 instancias
Clase 3: 659 instancias

Classification Report:
              precision    recall  f1-score   support

           1       0.00      0.00      0.00       160
           2       0.56      0.01      0.02       395
           3       0.17      0.99      0.29       114

    accuracy                           0.18       669
   macro avg       0.24      0.33      0.11       669
weighted avg       0.36      0.18      0.06       669





---



## **MODEL: sbcBI/sentiment_analysis_model**

https://huggingface.co/sbcBI/sentiment_analysis_model?text=im+happy%2C+very+happy

In [ ]:
from transformers import pipeline, AutoTokenizer

# Definir el modelo y el tokenizador
model_name = "sbcBI/sentiment_analysis_model"
tokenizer = AutoTokenizer.from_pretrained(model_name)
sentiment_task = pipeline("sentiment-analysis", model=model_name, tokenizer=tokenizer, truncation=True, max_length=512)


tokenizer_config.json:   0%|          | 0.00/333 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/769 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/268M [00:00<?, ?B/s]

In [ ]:
# Convertir cada texto en el formato deseado y hacer las predicciones
predictions_57 = []
x = 0
for i, text in enumerate(formatted_texts_all_text):
    try:
        # Hacer la predicción con el texto
        output_57 = sentiment_task(text)
        # Obtener la etiqueta de la predicción
        label = output_57[0]['label']
        # Convertir la etiqueta a su valor numérico
        if label == 'LABEL_2':
            predictions_57.append(1)
        elif label == 'LABEL_1':
            predictions_57.append(2)
        else:
            predictions_57.append(3)
        print(f"Texto {x}: {output_57}")
    except Exception as e:
        print(f"Error al procesar texto {text}: {e}")
        # Asignar un valor de 3 a la predicción en caso de error
        predictions_57.append(3)
    x += 1

print(predictions_5)


Texto 0: [{'label': 'LABEL_2', 'score': 0.8655475378036499}]
Texto 1: [{'label': 'LABEL_2', 'score': 0.8919129371643066}]
Texto 2: [{'label': 'LABEL_0', 'score': 0.939284086227417}]
Texto 3: [{'label': 'LABEL_1', 'score': 0.4817398488521576}]
Texto 4: [{'label': 'LABEL_0', 'score': 0.9111126661300659}]
Texto 5: [{'label': 'LABEL_0', 'score': 0.708537757396698}]
Texto 6: [{'label': 'LABEL_0', 'score': 0.8045473098754883}]
Texto 7: [{'label': 'LABEL_0', 'score': 0.9478155374526978}]
Texto 8: [{'label': 'LABEL_0', 'score': 0.8119521141052246}]
Texto 9: [{'label': 'LABEL_0', 'score': 0.7974138855934143}]
Texto 10: [{'label': 'LABEL_2', 'score': 0.9196903705596924}]
Texto 11: [{'label': 'LABEL_2', 'score': 0.9439244866371155}]
Texto 12: [{'label': 'LABEL_2', 'score': 0.6739258170127869}]
Texto 13: [{'label': 'LABEL_0', 'score': 0.4736577868461609}]
Texto 14: [{'label': 'LABEL_1', 'score': 0.3567350506782532}]
Texto 15: [{'label': 'LABEL_0', 'score': 0.48605746030807495}]
Texto 16: [{'label'

In [ ]:
from sklearn.metrics import classification_report

# Obtener las etiquetas originales del conjunto de datos
y_true = data['label'].iloc[-669:]
print(y_true)

# Calcular el conteo de cada clase
class_counts = {label: predictions_57.count(label) for label in set(predictions_57)}

# Mostrar el conteo de cada clase
print("Conteo de cada clase:")
for label, count in class_counts.items():
    print(f"Clase {label}: {count} instancias")

# Crear el classification_report con las etiquetas originales del DataFrame 'data'
print("\nClassification Report:")
print(classification_report(y_true, predictions_57))

1131    1
1132    1
1133    1
1134    1
1135    1
       ..
1795    3
1796    3
1797    3
1798    3
1799    3
Name: label, Length: 669, dtype: int64
Conteo de cada clase:
Clase 1: 130 instancias
Clase 2: 23 instancias
Clase 3: 516 instancias

Classification Report:
              precision    recall  f1-score   support

           1       0.42      0.34      0.38       160
           2       0.52      0.03      0.06       395
           3       0.21      0.96      0.35       114

    accuracy                           0.26       669
   macro avg       0.39      0.44      0.26       669
weighted avg       0.45      0.26      0.18       669



---



## **MODEL: Sigma/financial-sentiment-analysis**

https://huggingface.co/Sigma/financial-sentiment-analysis?text=Something+that+work+for+me+is+to+expose+myself+to+my+anxiety.+Even+though+I%27m+starting+to+become+anxious%2C+I+keep+doing+it.+The+trick+is+to+not+go+over+a+threshold.+If+I%27m+too+anxious%2C+it+affect+how+I+act+and+then+everything+goes+bad+%28not+that+bad+but+my+anxiety+amplify+everything+that+goes+bad+and+then+I+keep+thinking+about+it+and+it+just+worse%29.+Taking+a+walk+is+a+great+way%2C+you+can+start+in+a+residential+place+with+less+people+and+then+move+on+to+place+with+more+and+more.+Going+to+movie+theaters%2C+fast-food%2C+restaurants%2C+conventions%2C+also+helped+me.+All+situations+where+I+had+just+a+little+anxiety+but+that+let+me+see+how+people+acts+and+let+me+understands+them+a+little+better.+Another+thing+that+you+need+to+do%2C+is+stopping+to+be+negative+about+yourself.+You+don%27t+seems+to+be+that+negative%2C+though+you+do+say+%22anyone+who+bothers+to+read+this%22+which+seems+like+it%27s+negative+reading+your+thread+%28it+doesn%27t+bother+us%2C+we+are+there+to+read+theses+threads%2C+like+any+other+one%29%2C+but+it%27s+important+that+you+are+aware+that+you+should+avoid+negative+though.+Often+we+justify+being+negative+by+saying+that+this+way+you+are+aware+of+your+flaw%2C+which+is+better+than+not.+Sadly+often+we+are+wrong%2C+theses+negatives+though+aren%27t+that+important+and+they+are+amazing+to+lose+any+motivation+to+improve.+Instead%2C+you+need+to+find+the+positive+in+the+situation+and+if+really+there%27s+something+negative+important%2C+don%27t+see+it+as+negative+but+as+something+to+improve.++at+this+point+i+have+no+assurances+and+putting+my+faith+in+a+psychologist+only+to+go+thru+months+and+years+of+no+improvement+would+probably+destroy+me.+It+doesn%27t+destroy+you+right+now%3F+Really+how+trying+anything+is+worst+than+not+trying+anything+and+staying+the+mess+you+are+right+now%3F+Believe+me%2C+you+can%27t+be+worst+than+you+are+right+now%2C+you+can+fail%2C+and+you+may+fail+often%2C+but+that+will+just+bring+you+back+to+where+you+are+right+now.+Anxiety+is+a+bitch+though+and+it+will+always+try+to+give+you+the+impression+you+don%27t+improve+and+are+just+worst+than+you+ever+was+by+simply+trying.+You+need+to+stay+positive.+Therapist+know+what+you+live%2C+they+know+plenty+of+ways+of+dealing+with+it%2C+they+are+the+professional%2C+they+know+what+they+are+doing.+I+never+saw+one+because+I%27m+sure+I+can+fix+that+by+myself%2C+I+hate+depending+on+someone%2C+but+I+have+no+doubt+that+I+would+be+able+to+do+it+much+quicker+than+currently+with+their+help.+I+already+convinced+myself+to+go+see+one+if+I+no+longer+see+improvement.

In [ ]:
from transformers import pipeline, AutoTokenizer

# Definir el modelo y el tokenizador
model_name = "Sigma/financial-sentiment-analysis"
tokenizer = AutoTokenizer.from_pretrained(model_name)
sentiment_task = pipeline("sentiment-analysis", model=model_name, tokenizer=tokenizer, truncation=True, max_length=512)

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/854 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/439M [00:00<?, ?B/s]

In [ ]:
# Convertir cada texto en el formato deseado y hacer las predicciones
predictions_58 = []
x = 0
for i, text in enumerate(formatted_texts_all_text):
    try:
        # Hacer la predicción con el texto
        output_58 = sentiment_task(text)
        # Obtener la etiqueta de la predicción
        label = output_58[0]['label']
        # Convertir la etiqueta a su valor numérico
        if label == 'positive':
            predictions_58.append(1)
        elif label == 'neutral':
            predictions_58.append(2)
        else:
            predictions_58.append(3)
        print(f"Texto {x}: {output_58}")
    except Exception as e:
        print(f"Error al procesar texto {text}: {e}")
        # Asignar un valor de 3 a la predicción en caso de error
        predictions_58.append(3)
    x += 1

print(predictions_58)


Texto 0: [{'label': 'positive', 'score': 0.6255910396575928}]
Texto 1: [{'label': 'positive', 'score': 0.9262599349021912}]
Texto 2: [{'label': 'neutral', 'score': 0.48628246784210205}]
Texto 3: [{'label': 'neutral', 'score': 0.615001916885376}]
Texto 4: [{'label': 'neutral', 'score': 0.5945963263511658}]
Texto 5: [{'label': 'negative', 'score': 0.46583807468414307}]
Texto 6: [{'label': 'neutral', 'score': 0.5189802646636963}]
Texto 7: [{'label': 'negative', 'score': 0.7029042840003967}]
Texto 8: [{'label': 'neutral', 'score': 0.5460911393165588}]
Texto 9: [{'label': 'negative', 'score': 0.9399840235710144}]
Texto 10: [{'label': 'positive', 'score': 0.965195894241333}]
Texto 11: [{'label': 'positive', 'score': 0.8717069029808044}]
Texto 12: [{'label': 'neutral', 'score': 0.4816897213459015}]
Texto 13: [{'label': 'neutral', 'score': 0.5801571011543274}]
Texto 14: [{'label': 'positive', 'score': 0.4648309648036957}]
Texto 15: [{'label': 'neutral', 'score': 0.6382167935371399}]
Texto 16: 

In [ ]:
from sklearn.metrics import classification_report

# Obtener las etiquetas originales del conjunto de datos
y_true = data['label'].iloc[-669:]
print(y_true)

# Calcular el conteo de cada clase
class_counts = {label: predictions_58.count(label) for label in set(predictions_58)}

# Mostrar el conteo de cada clase
print("Conteo de cada clase:")
for label, count in class_counts.items():
    print(f"Clase {label}: {count} instancias")

# Crear el classification_report con las etiquetas originales del DataFrame 'data'
print("\nClassification Report:")
print(classification_report(y_true, predictions_58))

1131    1
1132    1
1133    1
1134    1
1135    1
       ..
1795    3
1796    3
1797    3
1798    3
1799    3
Name: label, Length: 669, dtype: int64
Conteo de cada clase:
Clase 1: 94 instancias
Clase 2: 260 instancias
Clase 3: 315 instancias

Classification Report:
              precision    recall  f1-score   support

           1       0.52      0.31      0.39       160
           2       0.56      0.37      0.45       395
           3       0.26      0.72      0.38       114

    accuracy                           0.41       669
   macro avg       0.45      0.47      0.40       669
weighted avg       0.50      0.41      0.42       669




---



## **MODEL: MarieAngeA13/Sentiment-Analysis-BERT**

https://huggingface.co/MarieAngeA13/Sentiment-Analysis-BERT?text=Something+that+work+for+me+is+to+expose+myself+to+my+anxiety.+Even+though+I%27m+starting+to+become+anxious%2C+I+keep+doing+it.+The+trick+is+to+not+go+over+a+threshold.+If+I%27m+too+anxious%2C+it+affect+how+I+act+and+then+everything+goes+bad+%28not+that+bad+but+my+anxiety+amplify+everything+that+goes+bad+and+then+I+keep+thinking+about+it+and+it+just+worse%29.+Taking+a+walk+is+a+great+way%2C+you+can+start+in+a+residential+place+with+less+people+and+then+move+on+to+place+with+more+and+more.+Going+to+movie+theaters%2C+fast-food%2C+restaurants%2C+conventions%2C+also+helped+me.+All+situations+where+I+had+just+a+little+anxiety+but+that+let+me+see+how+people+acts+and+let+me+understands+them+a+little+better.+Another+thing+that+you+need+to+do%2C+is+stopping+to+be+negative+about+yourself.+You+don%27t+seems+to+be+that+negative%2C+though+you+do+say+%22anyone+who+bothers+to+read+this%22+which+seems+like+it%27s+negative+reading+your+thread+%28it+doesn%27t+bother+us%2C+we+are+there+to+read+theses+threads%2C+like+any+other+one%29%2C+but+it%27s+important+that+you+are+aware+that+you+should+avoid+negative+though.+Often+we+justify+being+negative+by+saying+that+this+way+you+are+aware+of+your+flaw%2C+which+is+better+than+not.+Sadly+often+we+are+wrong%2C+theses+negatives+though+aren%27t+that+important+and+they+are+amazing+to+lose+any+motivation+to+improve.+Instead%2C+you+need+to+find+the+positive+in+the+situation+and+if+really+there%27s+something+negative+important%2C+don%27t+see+it+as+negative+but+as+something+to+improve.++at+this+point+i+have+no+assurances+and+putting+my+faith+in+a+psychologist+only+to+go+thru+months+and+years+of+no+improvement+would+probably+destroy+me.+It+doesn%27t+destroy+you+right+now%3F+Really+how+trying+anything+is+worst+than+not+trying+anything+and+staying+the+mess+you+are+right+now%3F+Believe+me%2C+you+can%27t+be+worst+than+you+are+right+now%2C+you+can+fail%2C+and+you+may+fail+often%2C+but+that+will+just+bring+you+back+to+where+you+are+right+now.+Anxiety+is+a+bitch+though+and+it+will+always+try+to+give+you+the+impression+you+don%27t+improve+and+are+just+worst+than+you+ever+was+by+simply+trying.+You+need+to+stay+positive.

In [ ]:
from transformers import pipeline, AutoTokenizer

# Definir el modelo y el tokenizador
model_name = "MarieAngeA13/Sentiment-Analysis-BERT"
tokenizer = AutoTokenizer.from_pretrained(model_name)
sentiment_task = pipeline("sentiment-analysis", model=model_name, tokenizer=tokenizer, truncation=True, max_length=512)

tokenizer_config.json:   0%|          | 0.00/313 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/944 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/438M [00:00<?, ?B/s]

In [ ]:
# Convertir cada texto en el formato deseado y hacer las predicciones
predictions_59 = []
x = 0
for i, text in enumerate(formatted_texts_all_text):
    try:
        # Hacer la predicción con el texto
        output_59 = sentiment_task(text)
        # Obtener la etiqueta de la predicción
        label = output_59[0]['label']
        # Convertir la etiqueta a su valor numérico
        if label == 'positive':
            predictions_59.append(1)
        elif label == 'neutral':
            predictions_59.append(2)
        else:
            predictions_59.append(3)
        print(f"Texto {x}: {output_59}")
    except Exception as e:
        print(f"Error al procesar texto {text}: {e}")
        # Asignar un valor de 3 a la predicción en caso de error
        predictions_59.append(3)
    x += 1

print(predictions_59)


Texto 0: [{'label': 'positive', 'score': 0.6255910396575928}]
Texto 1: [{'label': 'positive', 'score': 0.9262599349021912}]
Texto 2: [{'label': 'neutral', 'score': 0.48628246784210205}]
Texto 3: [{'label': 'neutral', 'score': 0.615001916885376}]
Texto 4: [{'label': 'neutral', 'score': 0.5945963263511658}]
Texto 5: [{'label': 'negative', 'score': 0.46583807468414307}]
Texto 6: [{'label': 'neutral', 'score': 0.5189802646636963}]
Texto 7: [{'label': 'negative', 'score': 0.7029042840003967}]
Texto 8: [{'label': 'neutral', 'score': 0.5460911393165588}]
Texto 9: [{'label': 'negative', 'score': 0.9399840235710144}]
Texto 10: [{'label': 'positive', 'score': 0.965195894241333}]
Texto 11: [{'label': 'positive', 'score': 0.8717069029808044}]
Texto 12: [{'label': 'neutral', 'score': 0.4816897213459015}]
Texto 13: [{'label': 'neutral', 'score': 0.5801571011543274}]
Texto 14: [{'label': 'positive', 'score': 0.4648309648036957}]
Texto 15: [{'label': 'neutral', 'score': 0.6382167935371399}]
Texto 16: 

In [ ]:
from sklearn.metrics import classification_report

# Obtener las etiquetas originales del conjunto de datos
y_true = data['label'].iloc[-669:]
print(y_true)

# Calcular el conteo de cada clase
class_counts = {label: predictions_59.count(label) for label in set(predictions_59)}

# Mostrar el conteo de cada clase
print("Conteo de cada clase:")
for label, count in class_counts.items():
    print(f"Clase {label}: {count} instancias")

# Crear el classification_report con las etiquetas originales del DataFrame 'data'
print("\nClassification Report:")
print(classification_report(y_true, predictions_59))

1131    1
1132    1
1133    1
1134    1
1135    1
       ..
1795    3
1796    3
1797    3
1798    3
1799    3
Name: label, Length: 669, dtype: int64
Conteo de cada clase:
Clase 1: 94 instancias
Clase 2: 260 instancias
Clase 3: 315 instancias

Classification Report:
              precision    recall  f1-score   support

           1       0.52      0.31      0.39       160
           2       0.56      0.37      0.45       395
           3       0.26      0.72      0.38       114

    accuracy                           0.41       669
   macro avg       0.45      0.47      0.40       669
weighted avg       0.50      0.41      0.42       669





---



## **MODEL: cardiffnlp/twitter-xlm-roberta-base-sentiment**

https://huggingface.co/cardiffnlp/twitter-xlm-roberta-base-sentiment?text=Something+that+work+for+me+is+to+expose+myself+to+my+anxiety.+Even+though+I%27m+starting+to+become+anxious%2C+I+keep+doing+it.+The+trick+is+to+not+go+over+a+threshold.+If+I%27m+too+anxious%2C+it+affect+how+I+act+and+then+everything+goes+bad+%28not+that+bad+but+my+anxiety+amplify+everything+that+goes+bad+and+then+I+keep+thinking+about+it+and+it+just+worse%29.+Taking+a+walk+is+a+great+way%2C+you+can+start+in+a+residential+place+with+less+people+and+then+move+on+to+place+with+more+and+more.+Going+to+movie+theaters%2C+fast-food%2C+restaurants%2C+conventions%2C+also+helped+me.+All+situations+where+I+had+just+a+little+anxiety+but+that+let+me+see+how+people+acts+and+let+me+understands+them+a+little+better.+Another+thing+that+you+need+to+do%2C+is+stopping+to+be+negative+about+yourself.+You+don%27t+seems+to+be+that+negative%2C+though+you+do+say+%22anyone+who+bothers+to+read+this%22+which+seems+like+it%27s+negative+reading+your+thread+%28it+doesn%27t+bother+us%2C+we+are+there+to+read+theses+threads%2C+like+any+other+one%29%2C+but+it%27s+important+that+you+are+aware+that+you+should+avoid+negative+though.+Often+we+justify+being+negative+by+saying+that+this+way+you+are+aware+of+your+flaw%2C+which+is+better+than+not.+Sadly+often+we+are+wrong%2C+theses+negatives+though+aren%27t+that+important+and+they+are+amazing+to+lose+any+motivation+to+improve.+Instead%2C+you+need+to+find+the+positive+in+the+situation+and+if+really+there%27s+something+negative+important%2C+don%27t+see+it+as+negative+but+as+something+to+improve.++at+this+point+i+have+no+assurances+and+putting+my+faith+in+a+psychologist+only+to+go+thru+months+and+years+of+no+improvement+would+probably+destroy+me.+It+doesn%27t+destroy+you+right+now%3F+Really+how+trying+anything+is+worst+than+not+trying+anything+and+staying+the+mess+you+are+right+now%3F+Believe+me%2C+you+can%27t+be+worst+than+you+are+right+now%2C+you+can+fail%2C+and+you+may+fail+often%2C+but+that+will+just+bring+you+back+to+where+you+are+right+now.+Anxiety+is+a+bitch+though+and+it+will+always+try+to+give+you+the+impression+you+don%27t+improve+and+are+just+worst+than+you+ever+was+by+simply+trying.+You+need+to+stay+positive.

In [ ]:
from transformers import pipeline, AutoTokenizer

# Definir el modelo y el tokenizador
model_name = "cardiffnlp/twitter-xlm-roberta-base-sentiment"
tokenizer = AutoTokenizer.from_pretrained(model_name)
sentiment_task = pipeline("sentiment-analysis", model=model_name, tokenizer=tokenizer, truncation=True, max_length=512)


config.json:   0%|          | 0.00/841 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/150 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/1.11G [00:00<?, ?B/s]

In [ ]:
# Convertir cada texto en el formato deseado y hacer las predicciones
predictions_60 = []
x = 0
for i, text in enumerate(formatted_texts_all_text):
    try:
        # Hacer la predicción con el texto
        output_60 = sentiment_task(text)
        # Obtener la etiqueta de la predicción
        label = output_60[0]['label']
        # Convertir la etiqueta a su valor numérico
        if label == 'positive':
            predictions_60.append(1)
        elif label == 'neutral':
            predictions_60.append(2)
        else:
            predictions_60.append(3)
        print(f"Texto {x}: {output_60}")
    except Exception as e:
        print(f"Error al procesar texto {text}: {e}")
        # Asignar un valor de 3 a la predicción en caso de error
        predictions_60.append(3)
    x += 1

print(predictions_60)


Texto 0: [{'label': 'negative', 'score': 0.4656684994697571}]
Texto 1: [{'label': 'positive', 'score': 0.7422499060630798}]
Texto 2: [{'label': 'negative', 'score': 0.6556121706962585}]
Texto 3: [{'label': 'negative', 'score': 0.5650849938392639}]
Texto 4: [{'label': 'negative', 'score': 0.8402629494667053}]
Texto 5: [{'label': 'neutral', 'score': 0.3820309340953827}]
Texto 6: [{'label': 'negative', 'score': 0.504920244216919}]
Texto 7: [{'label': 'negative', 'score': 0.4773251712322235}]
Texto 8: [{'label': 'negative', 'score': 0.4801366627216339}]
Texto 9: [{'label': 'negative', 'score': 0.7766801118850708}]
Texto 10: [{'label': 'positive', 'score': 0.943144679069519}]
Texto 11: [{'label': 'positive', 'score': 0.811928927898407}]
Texto 12: [{'label': 'neutral', 'score': 0.4014851748943329}]
Texto 13: [{'label': 'negative', 'score': 0.4530172348022461}]
Texto 14: [{'label': 'neutral', 'score': 0.467810720205307}]
Texto 15: [{'label': 'negative', 'score': 0.4491540193557739}]
Texto 16:

In [ ]:
from sklearn.metrics import classification_report

# Obtener las etiquetas originales del conjunto de datos
y_true = data['label'].iloc[-669:]
print(y_true)

# Calcular el conteo de cada clase
class_counts = {label: predictions_60.count(label) for label in set(predictions_60)}

# Mostrar el conteo de cada clase
print("Conteo de cada clase:")
for label, count in class_counts.items():
    print(f"Clase {label}: {count} instancias")

# Crear el classification_report con las etiquetas originales del DataFrame 'data'
print("\nClassification Report:")
print(classification_report(y_true, predictions_60))

1131    1
1132    1
1133    1
1134    1
1135    1
       ..
1795    3
1796    3
1797    3
1798    3
1799    3
Name: label, Length: 669, dtype: int64
Conteo de cada clase:
Clase 1: 62 instancias
Clase 2: 84 instancias
Clase 3: 523 instancias

Classification Report:
              precision    recall  f1-score   support

           1       0.63      0.24      0.35       160
           2       0.62      0.13      0.22       395
           3       0.21      0.97      0.35       114

    accuracy                           0.30       669
   macro avg       0.49      0.45      0.31       669
weighted avg       0.55      0.30      0.27       669

